In [8]:
import os
import json
import pandas as pd
import traceback
import PyPDF2
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
from langchain.callbacks import get_openai_callback

In [9]:
from langchain.chat_models import ChatOpenAI
from langchain_huggingface import HuggingFaceEndpoint

We have to install the python dotenv dependency and also, we have to call its class before we can activate our .env variables

In [70]:
from dotenv import load_dotenv

load_dotenv()

True

In [71]:
KEY = os.getenv("OPENAI_API_KEY")

In [72]:
os.environ["OPENAI_API_TOKEN"] = KEY

In [74]:
# llm = HuggingFaceEndpoint(repo_id="google/flan-t5-large", huggingfacehub_api_token=KEY)
llm = ChatOpenAI(openai_api_key=KEY,model_name="gpt-3.5-turbo", temperature=0.5)

In [52]:
# llm

In [13]:
RESPONSE_JSON = {
    "1": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "2": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "3": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
}

In [14]:
TEMPLATE = """
Text:{text}
You are an expert MCQ maker. Given the above text, it is your job to \
create a quiz  of {number} multiple choice questions for {subject} students in {tone} tone. 
Make sure the questions are not repeated and check all the questions to be conforming the text as well.
Make sure to format your response like  RESPONSE_JSON below  and use it as a guide. \
Ensure to make {number} MCQs
### RESPONSE_JSON
{response_json}
"""

In [15]:
quiz_generation_prompt = PromptTemplate(
    input_variables=['text', 'number', 'subject', 'tone', 'response_json'],
    template=TEMPLATE
)

In [75]:
quiz_chain = LLMChain(llm=llm, prompt=quiz_generation_prompt, output_key="quiz", verbose=True)

In [17]:
TEMPLATE2 = """
You are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} students.\
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis. 
if the quiz is not at per with the cognitive and analytical abilities of the students,\
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities
Quiz_MCQs:
{quiz}

Check from an expert English Writer of the above quiz:
"""

In [20]:
quiz_evaluation_prompt = PromptTemplate(input_variables=["subject", "quiz"], template=TEMPLATE2)

In [76]:
review_chain = LLMChain(llm=llm, prompt=quiz_evaluation_prompt, output_key="review", verbose=True)

In [55]:
generate_evaluate_chain = SequentialChain(chains=[quiz_chain, review_chain], input_variables=["text", "number", "subject", "tone", "response_json"], output_variables=["quiz", "review"], verbose=True,)

r means read it in the beginning of the file path

In [26]:
file_path = r"C:\Users\User\Data Science\Deep Learning\Generative AI\MCQGEN\data.txt"

In [27]:
file_path

'C:\\Users\\User\\Data Science\\Deep Learning\\Generative AI\\MCQGEN\\data.txt'

In [38]:
with open(file_path, 'r') as file:
    TEXT = file.read()

In [39]:
print(TEXT)

The term machine learning was coined in 1959 by Arthur Samuel, an IBM employee and pioneer in the field of computer gaming and artificial intelligence.[9][10] The synonym self-teaching computers was also used in this time period.[11][12]\n\nAlthough the earliest machine learning model was introduced in the 1950s when Arthur Samuel invented a program that calculated the winning chance in checkers for each side, the history of machine learning roots back to decades of human desire and effort to study human cognitive processes.[13] In 1949, Donald Hebb, a Canadian psychologist, published a book titled The Organization of Behavior where he introduces the Hebbian theory, discussing the neural structure or synapses between the nerve cells.[14] Hebbâ€™s model of neurons interacting with one another sets a groundwork for how AIs and machine learning algorithms work under nodes, or artificial neurons used by computers to communicate data.[13] Other researchers who have studied on human cognitiv

Serialize the Python dictionary into a JSON-formatted string

In [40]:
json.dumps(RESPONSE_JSON)

'{"1": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "2": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "3": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}}'

In [31]:
NUMBER = 5
SUBJECT = "biology"
TONE = "simple"

https://python.langchain.com/docs/modules/model_io/llms/token_usage_tracking

How to setup Token Usage Tracking in LangChain

In [77]:
with get_openai_callback() as cb:
    response = generate_evaluate_chain(
        {
            "text": TEXT,
            "number": NUMBER,
            "subject": SUBJECT,
            "tone": TONE,
            "response_json": json.dumps(RESPONSE_JSON)
        }
    )



> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Text:The term machine learning was coined in 1959 by Arthur Samuel, an IBM employee and pioneer in the field of computer gaming and artificial intelligence.[9][10] The synonym self-teaching computers was also used in this time period.[11][12]\n\nAlthough the earliest machine learning model was introduced in the 1950s when Arthur Samuel invented a program that calculated the winning chance in checkers for each side, the history of machine learning roots back to decades of human desire and effort to study human cognitive processes.[13] In 1949, Donald Hebb, a Canadian psychologist, published a book titled The Organization of Behavior where he introduces the Hebbian theory, discussing the neural structure or synapses between the nerve cells.[14] Hebbâ€™s model of neurons interacting with one another sets a groundwork for how AIs and machine learning algorithms work under nodes, or artifi

RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}

In [36]:
# print(f"Total Tokens:{cb.total_tokens}")
# print(f"Prompt Tokens:{cb.prompt_tokens}")
# print(f"Completion Tokens:{cb.completion_tokens}")
# print(f"Total Cost:{cb.total_cost}")

In [41]:
response

NameError: name 'response' is not defined

In [ ]:
quiz = response.get("quiz")

In [ ]:
quiz = json.loads(quiz)

In [ ]:
quiz_table_data = []
for key, value in quiz.items():
    mcq = value["mcq"]
    options = " | ".join(
        [
            f"{option}: {option_value}"
            for option, option_value in value["options"].items()
        ]
    )
    correct = value["correct"]
    quiz_table_data.append(
        {"MCQ": mcq, "Choices": options, "Correct": correct})

In [ ]:
quiz_table_data

In [ ]:
quiz = pd.DataFrame(quiz_table_data)

In [ ]:
quiz.to_csv("machinelearning.csv", index=False)

In [ ]:
from datetime import datetime
datetime.now().strftime('%m_%d_%Y_%H_%M_%S')